In [2]:
from rdflib import Graph

In [4]:
uri = "http://id.loc.gov/resources/instances/1893399"
graph = Graph()
graph.parse(f'{uri}.rdf')
# graph.serialize("instance.ttl")

<Graph identifier=N2700e5b007ff4d11a5bd8b8c77f33c74 (<class 'rdflib.graph.Graph'>)>

In [107]:
from pydantic import BaseModel
from typing import Optional
from api.src.schemas.metadata.bibframe.adminMetadata import AdminMetadata
from api.src.schemas.metadata.bibframe.title import Title
from api.src.schemas.metadata.bibframe.element import Element
# from src.schemas.metadata.bibframe.contribution import Contribution

class Valeu(BaseModel):
    label: str
    type: str

class ProvisionActivity(BaseModel):
    agent: Optional[str]
    date: str
    place: str

class Instance(BaseModel):
    adminMetadata: AdminMetadata
    type: list[str]
    title: Title
    carrier: list[Element]
    copyrightDate: Optional[str]
    dimensions: Optional[str]
    extent: Optional[Valeu]
    instanceOf: list[Element]
    issuance: Optional[list[Element]]
    media: Optional[list[Element]]
    provisionActivityStatement: Optional[str]
    responsibilityStatement: Optional[str]
    seriesStatement: Optional[str]
    # content: list[Element]
    # language: list[Element]
    # 
    # 
    # contribution: Optional[list[Contribution]]
    # subject: Optional[list[Element]]   
    # genreForm: Optional[list[Element]]
    # note: Optional[str]
    # summary: Optional[str]
    # tableOfContents: Optional[str]
    # supplementaryContent: Optional[list[Element]]
    # illustrativeContent: Optional[list[Element]]
    # intendedAudience: Optional[list[Element]]
    # geographicCoverage: Optional[list[Element]]

In [116]:
from api.src.function.bibframe.bf_type import GetType
from api.src.function.bibframe.bf_title import GetTitle
from api.src.function.bibframe.bf_Uri import GetUriBF
# from src.function.bibframe.bf_contribution import GetContribution
# 
# from src.function.bibframe.bf_Literal import GetLiteral

def GetElement(graph, bf, obj):
    q = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?type ?label
                WHERE {{ 
                    <{uri}> bf:{bf} ?uri .
                    ?uri rdfs:label ?label .
                    ?uri rdf:type ?type .
                    }}"""
    response = graph.query(q)
    bindings = response.bindings
    if len(bindings) > 0:
        binding = bindings[0]
        v = {
            "type": binding.get('type').toPython(),
            "label": binding.get('label').toPython()
        }
        obj[bf] = v
    return obj

def GetValeu(graph, bf, obj):
    q = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?value
                WHERE {{ 
                    <{uri}> bf:{bf} ?value .
                    }}"""
    response = graph.query(q)
    bindings = response.bindings
    if len(bindings) > 0:
        binding = bindings[0]
        obj[bf] = binding.get('value').toPython()
    return obj

def ParserInstance(graph, uri):
    identifier = uri.split("/")[-1]
    types = GetType(graph, uri)
    title = GetTitle(graph, uri)

    obj = {
    'adminMetadata': {
        'generationProcess': {
            'label': 'BiblioKeia'
        },
        'identifiedBy': [ 
            {
                "type": "Local",
            "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
            "value": identifier
            }
        ]
    },
    "type": types,
    'title': title 
    }
    # carrier
    obj = GetUriBF(graph, uri, 'carrier', obj)
    # copyrightDate
    # obj = GetValeu(graph, "copyrightDate", obj)
    # dimensions
    obj = GetValeu(graph, "dimensions", obj)
    # extent
    obj = GetElement(graph, "extent", obj)
    # instanceOf
    obj = GetUriBF(graph, uri, 'instanceOf', obj)
    # issuance
    obj = GetUriBF(graph, uri, 'issuance', obj)
    # media
    obj = GetUriBF(graph, uri, 'media', obj)
    # provisionActivityStatement
    obj = GetValeu(graph, "provisionActivityStatement", obj)
    # responsibilityStatement
    obj = GetValeu(graph, "responsibilityStatement", obj)
    # seriesStatement
    obj = GetValeu(graph, "seriesStatement", obj)




    # # Content
    # # obj = GetContent(graph, uri, obj)
    # # Contribution
    # obj = GetContribution(graph, uri, obj)
    # # Content
    # obj = GetUriBF(graph, uri, 'content', obj)
    # # GenreForm
    # obj = GetUriBF(graph, uri, 'genreForm', obj)
    # # illustrativeContent
    # obj = GetUriBF(graph, uri, 'illustrativeContent', obj)
    # # intendedAudience
    # obj = GetUriBF(graph, uri, 'intendedAudience', obj)
    # # language
    # obj = GetUriBF(graph, uri, 'language', obj)
    # # subject
    # obj = GetUriBF(graph, uri, 'subject', obj)
    # # supplementaryContent
    # obj = GetUriBF(graph, uri, 'supplementaryContent', obj)
    # # geographicCoverage
    # obj = GetUriBF(graph, uri, 'geographicCoverage', obj)
    # # summary
    # obj = GetLiteral(graph, uri, 'summary', obj)
    # # tableOfContents
    # obj = GetLiteral(graph, uri, 'tableOfContents', obj)

    response = Instance(**obj)
    
    return response

response = ParserInstance(graph, uri)
response.title

Title(type='bf:Title', mainTitle='Conjectures and refutations', subtitle=None)

In [89]:
obj = GetElement(graph, "instanceOf", {})
obj


{'instanceOf': {'type': 'http://id.loc.gov/ontologies/bibframe/Work',
  'label': 'Popper, Karl R. (Karl Raimund), 1902-1994. Conjectures and refutations : the growth of scientific knowledge'}}

In [110]:
q = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
              SELECT ?mainTitle 
                WHERE {{ 
                  <{uri}> bf:title ?title .
                ?title bf:mainTitle ?mainTitle
                  }}"""
response = graph.query(q)
[binding] = response.bindings
mainTitle = binding.get('mainTitle').value
title = {
    'mainTitle':mainTitle
  }

In [115]:
def GetTitle(graph, uri): 

  q = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
            SELECT ?mainTitle ?subtitle ?label
            WHERE {{ 
                <{uri}> bf:title ?title .
                 ?title rdfs:label ?label .
  ?title bf:mainTitle ?mainTitle .
  OPTIONAL {{ ?title bf:subtitle ?subtitle }}
                  }}"""
  response = graph.query(q)
  [binding] = response.bindings
  mainTitle = binding.get('mainTitle').value

  title = {
    'mainTitle':mainTitle,
    'subtitle': binding.get('subtitle').value,
    'label': binding.get('label').value,
  }
  
  return title
binding = GetTitle(graph, uri)
binding

{'mainTitle': 'Conjectures and refutations',
 'subtitle': 'the growth of scientific knowledge',
 'label': 'Conjectures and refutations : the growth of scientific knowledge'}

In [98]:
bindings = response.bindings
for binding in bindings:
    print(binding.get('o'))

http://id.loc.gov/ontologies/bibframe/ProvisionActivity
http://id.loc.gov/ontologies/bibframe/Publication
1968
http://id.loc.gov/vocabulary/countries/nyu
http://id.loc.gov/ontologies/bibframe/ProvisionActivity
http://id.loc.gov/ontologies/bibframe/Publication
Nae282dd406004023a42cfe3b42438275
Ne172f9407310411a91342c79ccc29be4
1968, c1965


In [49]:
t = binding.get('type').toPython()
t

'http://id.loc.gov/ontologies/bibframe/Extent'

In [100]:
graph.triples(())

TypeError: Graph.triples() missing 1 required positional argument: 'triple'